# Задание 1: Модификация существующих моделей (30 баллов)

1.1 Расширение линейной регрессии (15 баллов)
 <br>Модифицируйте существующую линейную регрессию:<br>
 - Добавьте L1 и L2 регуляризацию
 - Добавьте early stopping

In [ ]:
import torch
import torch.nn as nn

class LinearModel(nn.Module):
    def __init__(self,in_features,out_features):
        super().__init__()
        self.layer1 = nn.Linear(in_features,out_features)
    
    def forward(self,x):
        return self.layer1(x)


#l1_loss 
def l1_loss(output, target, activations, criterion, l1_lambda):
    main_loss = criterion(output, target)
    l1_penalty = torch.norm(activations, p=1)
    return main_loss + l1_lambda * l1_penalty


#класс для ранней остановки
class EarlyStopping:
    """"
    patience-количество эпох, которые нужно подождать, прежде чем остановиться, если улучшения не будет.
    delta-Minimum change in the monitored quantity to qualify as an improvement.
    best_score,best_model_state -Track the best validation score and model state.

    
    """
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_model_state = None

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model_state = model.state_dict()
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_state = model.state_dict()
            self.counter = 0

    def load_best_model(self, model):
        model.load_state_dict(self.best_model_state)

Пример обучения со всеми добавлениями(L2-loss добавлю использую здесь т.к оно уже реализовано в оптимизаторах)

In [ ]:
import torch.optim as optim

model = LinearModel(10,1)

train_loader = ... #какая-то тренировачная выборка
val_loader = ... #какая-то валидационная выборка



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=0.01)#weight_decay и отвечает за L2-регуляризацию
early_stopping = EarlyStopping(patience=5, delta=0.01)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)

    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item() * data.size(0)

    val_loss /= len(val_loader.dataset)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

early_stopping.load_best_model(model)

<br>1.2 Расширение логистической регрессии (15 баллов)<br>
Модифицируйте существующую логистическую регрессию:
 - Добавьте поддержку многоклассовой классификации
 - Реализуйте метрики: precision, recall, F1-score, ROC-AUC
 - Добавьте визуализацию confusion matrix

Добавим поддержку многокласовой классификации с помощью количества нейронов на последнем слое равным количеству классов и с помощью функции потерь torch.nn.CrossEntropyLoss

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import os

class ClassificationMetrics:
    def __init__(self):
        """
        Инициализация класса метрик классификации
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def to_one_hot(self, labels, num_classes):
        """
        Преобразование меток в one-hot encoding
        
        Args:
            labels: тензор меток (batch_size,)
            num_classes: количество классов
            
        Returns:
            one-hot тензор (batch_size, num_classes)
        """
        return torch.nn.functional.one_hot(labels, num_classes).float()

    def precision(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики Precision
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            precision score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        
        if average == 'micro':
            true_positives = torch.sum(y_true == y_pred).float()
            predicted_positives = len(y_pred)
            return true_positives / predicted_positives if predicted_positives > 0 else 0
        
        precision_per_class = []
        weights = []
        
        for c in range(num_classes):
            true_positives = torch.sum((y_true == c) & (y_pred == c)).float()
            predicted_positives = torch.sum(y_pred == c).float()
            
            precision = true_positives / predicted_positives if predicted_positives > 0 else 0
            precision_per_class.append(precision)
            
            if average == 'weighted':
                weights.append(torch.sum(y_true == c).float() / len(y_true))
        
        precision_per_class = torch.tensor(precision_per_class)
        
        if average == 'macro':
            return torch.mean(precision_per_class).item()
        elif average == 'weighted':
            return torch.sum(precision_per_class * torch.tensor(weights)).item()
        
        return precision_per_class.tolist()

    def recall(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики Recall
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            recall score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        
        if average == 'micro':
            true_positives = torch.sum(y_true == y_pred).float()
            actual_positives = len(y_true)
            return true_positives / actual_positives if actual_positives > 0 else 0
        
        recall_per_class = []
        weights = []
        
        for c in range(num_classes):
            true_positives = torch.sum((y_true == c) & (y_pred == c)).float()
            actual_positives = torch.sum(y_true == c).float()
            
            recall = true_positives / actual_positives if actual_positives > 0 else 0
            recall_per_class.append(recall)
            
            if average == 'weighted':
                weights.append(torch.sum(y_true == c).float() / len(y_true))
        
        recall_per_class = torch.tensor(recall_per_class)
        
        if average == 'macro':
            return torch.mean(recall_per_class).item()
        elif average == 'weighted':
            return torch.sum(recall_per_class * torch.tensor(weights)).item()
        
        return recall_per_class.tolist()

    def f1_score(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики F1-score
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            f1 score
        """
        precision = self.precision(y_true, y_pred, average)
        recall = self.recall(y_true, y_pred, average)
        
        if average in ['macro', 'weighted']:
            return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        elif average == 'micro':
            return precision
        
        f1_scores = []
        for p, r in zip(precision, recall):
            f1 = 2 * (p * r) / (p + r) if (p + r) > 0 else 0
            f1_scores.append(f1)
        return f1_scores

    def roc_auc(self, y_true, y_scores, multi_class='ovr'):
        """
        Вычисление метрики ROC-AUC
        
        Args:
            y_true: истинные метки (batch_size,)
            y_scores: вероятности предсказаний (batch_size, num_classes)
            multi_class: 'ovr' (one-vs-rest) или 'ovo' (one-vs-one)
            
        Returns:
            roc-auc score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).cpu().numpy()
        y_scores = torch.tensor(y_scores, dtype=torch.float).cpu().numpy()
        
        num_classes = y_scores.shape[1]
        y_true_one_hot = self.to_one_hot(torch.tensor(y_true), num_classes).cpu().numpy()
        
        auc_scores = []
        
        if multi_class == 'ovr':
            for i in range(num_classes):
                fpr, tpr, _ = roc_curve(y_true_one_hot[:, i], y_scores[:, i])
                auc_score = auc(fpr, tpr)
                auc_scores.append(auc_score)
            return np.mean(auc_scores)
        
        elif multi_class == 'ovo':
            auc_sum = 0
            n_pairs = 0
            for i in range(num_classes):
                for j in range(i+1, num_classes):
                    mask = np.logical_or(y_true == i, y_true == j)
                    y_true_binary = (y_true[mask] == i).astype(int)
                    y_scores_binary = y_scores[mask, i] / (y_scores[mask, i] + y_scores[mask, j])
                    fpr, tpr, _ = roc_curve(y_true_binary, y_scores_binary)
                    auc_sum += auc(fpr, tpr)
                    n_pairs += 1
            return auc_sum / n_pairs if n_pairs > 0 else 0
        
        return auc_scores

    def confusion_matrix(self, y_true, y_pred, plot=True, save_path=None):
        """
        Вычисление и визуализация confusion matrix с возможностью сохранения
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            plot: флаг для отображения визуализации
            save_path: путь для сохранения графика (например, 'confusion_matrix.png')
            
        Returns:
            confusion matrix
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        cm = torch.zeros((num_classes, num_classes), dtype=torch.long)
        
        for t, p in zip(y_true, y_pred):
            cm[t, p] += 1
        
        if plot:
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm.numpy(), annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            
            if save_path:
                # Создаём директорию, если она не существует
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                plt.savefig(save_path, dpi=300, bbox_inches='tight')
                print(f"Confusion matrix сохранена в {save_path}")
            
            plt.show()
            plt.close()
        
        return cm
